# Construa Sua Primeira Equipe de Agentes Inteligentes: Um Bot de Clima Progressivo com ADK

Este tutorial é uma extensão do [exemplo de Início Rápido](https://google.github.io/adk-docs/get-started/quickstart/) para o [Kit de Desenvolvimento de Agente (ADK)](https://google.github.io/adk-docs/get-started/). Agora, você está pronto para mergulhar mais fundo e construir um **sistema multiagente** mais sofisticado.

Embarcaremos na construção de uma **equipe de agentes Bot de Clima**, adicionando progressivamente recursos avançados a uma base simples. Começando com um único agente que pode consultar o clima, adicionaremos incrementalmente capacidades como:

*   Aproveitar diferentes modelos de IA (Gemini, GPT, Claude).
*   Projetar subagentes especializados para tarefas distintas (como saudações e despedidas).
*   Habilitar a delegação inteligente entre agentes.
*   Dar memória aos agentes usando o estado de sessão persistente.
*   Implementar barreiras de segurança cruciais usando callbacks.

**Por que uma Equipe de Bot de Clima?**

Este caso de uso, embora aparentemente simples, fornece uma base prática e compreensível para explorar conceitos centrais do ADK essenciais para a construção de aplicações agênticas complexas do mundo real. Você aprenderá como estruturar interações, gerenciar estados, garantir a segurança e orquestrar múltiplos "cérebros" de IA trabalhando em conjunto.

**O que é o ADK mesmo?**

Como lembrete, o ADK é um framework Python projetado para simplificar o desenvolvimento de aplicações impulsionadas por Modelos de Linguagem Grandes (LLMs). Ele oferece blocos de construção robustos para criar agentes que podem raciocinar, planejar, utilizar ferramentas, interagir dinamicamente com os usuários e colaborar de forma eficaz dentro de uma equipe.

**Neste tutorial avançado, você irá dominar:**

*   ✅ **Definição e Uso de Ferramentas:** Criar funções Python (`tools`) que concedem aos agentes habilidades específicas (como buscar dados) e instruir os agentes sobre como usá-las de forma eficaz.
*   ✅ **Flexibilidade Multi-LLM:** Configurar agentes para utilizar vários LLMs líderes (Gemini, GPT-4o, Claude Sonnet) via integração com o LiteLLM, permitindo que você escolha o melhor modelo para cada tarefa.
*   ✅ **Delegação e Colaboração de Agentes:** Projetar subagentes especializados e habilitar o roteamento automático (`auto flow`) de solicitações do usuário para o agente mais apropriado dentro de uma equipe.
*   ✅ **Estado de Sessão para Memória:** Utilizar `Session State` e `ToolContext` para permitir que os agentes se lembrem de informações entre os turnos da conversa, levando a interações mais contextuais.
*   ✅ **Barreiras de Segurança com Callbacks:** Implementar `before_model_callback` e `before_tool_callback` para inspecionar, modificar ou bloquear solicitações/uso de ferramentas com base em regras predefinidas, aumentando a segurança e o controle da aplicação.

**Expectativa do Estado Final:**

Ao concluir este tutorial, você terá construído um sistema funcional multiagente de Bot de Clima. Este sistema não apenas fornecerá informações sobre o tempo, mas também lidará com as gentilezas da conversação, lembrará a última cidade verificada e operará dentro de limites de segurança definidos, tudo orquestrado usando o ADK.

**Pré-requisitos:**

*   ✅ **Sólido conhecimento de programação em Python.**
*   ✅ **Familiaridade com Modelos de Linguagem Grandes (LLMs), APIs e o conceito de agentes.**
*   ❗ **Crucialmente: Conclusão do(s) tutorial(s) de Início Rápido do ADK ou conhecimento fundamental equivalente dos conceitos básicos do ADK (Agent, Runner, SessionService, uso básico de Tool).** Este tutorial se baseia diretamente nesses conceitos.
*   ✅ **Chaves de API** para os LLMs que você pretende usar (ex: Google AI Studio para Gemini, Plataforma OpenAI, Console Anthropic).

---
**Nota sobre o Ambiente de Execução:**

Este tutorial está estruturado para ambientes de notebook interativos como Google Colab, Colab Enterprise ou notebooks Jupyter. Por favor, tenha em mente o seguinte:

*   **Executando Código Assíncrono:** Ambientes de notebook lidam com código assíncrono de maneira diferente. Você verá exemplos usando `await` (adequado quando um loop de eventos já está em execução, comum em notebooks) ou `asyncio.run()` (frequentemente necessário ao rodar como um script `.py` autônomo ou em configurações específicas de notebook). Os blocos de código fornecem orientação para ambos os cenários.
*   **Configuração Manual de Runner/Session:** Os passos envolvem a criação explícita de instâncias de `Runner` e `SessionService`. Essa abordagem é mostrada porque lhe dá controle refinado sobre o ciclo de vida de execução do agente, gerenciamento de sessão e persistência de estado.

**Alternativa: Usando as Ferramentas Integradas do ADK (UI Web / CLI / Servidor de API)**

Se você prefere uma configuração que lida com o runner e o gerenciamento de sessão automaticamente usando as ferramentas padrão do ADK, pode encontrar o código equivalente estruturado para esse propósito [aqui](https://github.com/google/adk-docs/tree/main/examples/python/tutorial/agent_team/adk-tutorial). Essa versão é projetada para ser executada diretamente com comandos como `adk web` (para uma UI da web), `adk run` (para interação via CLI) ou `adk api_server` (para expor uma API). Por favor, siga as instruções do `README.md` fornecidas nesse recurso alternativo.

---
**Pronto para construir sua equipe de agentes? Vamos mergulhar de cabeça!**

In [ ]:
# @title Passo 0: Instalação e configuração
# Instale o ADK e o LiteLLM para suporte a vários modelos

!pip install google-adk -q
!pip install litellm -q

print("Instalação concluída.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 19.0 MB/s eta 0:00:00
Instalação concluída.


In [ ]:
# @title Importar bibliotecas necessárias
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # Para suporte a vários modelos
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # Para criar conteúdo/partes da mensagem

import warnings
# Ignorar todos os avisos
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Bibliotecas importadas.")

Bibliotecas importadas.


In [ ]:
# @title Configure as Chaves de API (Substitua pelas suas chaves reais!)

# --- IMPORTANTE: Substitua os placeholders pelas suas chaves de API reais ---
from google.colab import userdata

# Chave da API Gemini (Obtenha no Google AI Studio: https://aistudio.google.com/app/apikey)
# os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_DE_API_DO_GOOGLE" # <--- SUBSTITUA
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# [Opcional]
# Chave da API OpenAI (Obtenha na Plataforma OpenAI: https://platform.openai.com/api-keys)
# os.environ['OPENAI_API_KEY'] = 'SUA_CHAVE_DE_API_DA_OPENAI' # <--- SUBSTITUA

# [Opcional]
# Chave da API Anthropic (Obtenha no Console da Anthropic: https://console.anthropic.com/settings/keys)
# os.environ['ANTHROPIC_API_KEY'] = 'SUA_CHAVE_DE_API_DA_ANTHROPIC' # <--- SUBSTITUA


# --- Verificar Chaves (Checagem Opcional) ---
print("Chaves de API Definidas:")
print(f"Chave da API Google definida: {'Sim' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'SUA_CHAVE_DE_API_DO_GOOGLE' else 'Não (SUBSTITUA O PLACEHOLDER!)'}")
print(f"Chave da API OpenAI definida: {'Sim' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'SUA_CHAVE_DE_API_DA_OPENAI' else 'Não (SUBSTITUA O PLACEHOLDER!)'}")
print(f"Chave da API Anthropic definida: {'Sim' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'SUA_CHAVE_DE_API_DA_ANTHROPIC' else 'Não (SUBSTITUA O PLACEHOLDER!)'}")

# Configure o ADK para usar as chaves de API diretamente (não o Vertex AI para esta configuração multi-modelo)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Nota de Segurança:** A melhor prática é gerenciar as chaves de API de forma segura (por exemplo, usando Colab Secrets ou variáveis de ambiente) em vez de codificá-las diretamente no notebook. Substitua as strings de placeholder acima.

Chaves de API Definidas:
Chave da API Google definida: Sim
Chave da API OpenAI definida: Não (SUBSTITUA O PLACEHOLDER!)
Chave da API Anthropic definida: Não (SUBSTITUA O PLACEHOLDER!)


In [ ]:
# --- Defina Constantes de Modelo para facilitar o uso ---

# Mais modelos suportados podem ser referenciados aqui: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# Mais modelos suportados podem ser referenciados aqui: https://docs.litellm.ai/docs/providers/openai#openai-chat-completion-models
MODEL_GPT_4O = "openai/gpt-4.1" # Você também pode tentar: gpt-4.1-mini, gpt-4o etc.

# Mais modelos suportados podem ser referenciados aqui: https://docs.litellm.ai/docs/providers/anthropic
MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-20250514" # Você também pode tentar: claude-opus-4-20250514 , claude-3-7-sonnet-20250219 etc

print("\nAmbiente configurado.")


Ambiente configurado.


Com certeza! Aqui está a tradução do texto para o português, mantendo a formatação original:

---

## Passo 1: Seu Primeiro Agente - Consulta Básica de Clima

Vamos começar construindo o componente fundamental do nosso Bot de Clima: um único agente capaz de realizar uma tarefa específica – procurar informações sobre o clima. Isso envolve a criação de duas peças centrais:

1.  **Uma Ferramenta:** Uma função Python que equipa o agente com a *habilidade* de buscar dados meteorológicos.
2.  **Um Agente:** O "cérebro" de IA que entende a solicitação do usuário, sabe que possui uma ferramenta de clima e decide quando e como usá-la.

---
**1. Defina a Ferramenta (`get_weather`)**

No ADK, as **Ferramentas** (Tools) são os blocos de construção que dão aos agentes capacidades concretas além da simples geração de texto. Elas são tipicamente funções Python comuns que realizam ações específicas, como chamar uma API, consultar um banco de dados ou fazer cálculos.

Nossa primeira ferramenta fornecerá um relatório meteorológico *simulado* (mock). Isso nos permite focar na estrutura do agente sem a necessidade de chaves de API externas por enquanto. Mais tarde, você poderia facilmente trocar essa função simulada por uma que chame um serviço de clima real.
**Conceito-Chave: Docstrings são Cruciais!** A LLM do agente depende muito da **docstring** da função para entender:

*   *O que* a ferramenta faz.
*   *Quando* usá-la.
*   *Quais argumentos* ela requer (`city: str`).
*   *Que informação* ela retorna.
**Melhor Prática:** Escreva docstrings claras, descritivas e precisas para suas ferramentas. Isso é essencial para que a LLM use a ferramenta corretamente.

In [ ]:
# @title Defina a Ferramenta get_weather
def get_weather(city: str) -> dict:
  """Recupera o boletim meteorológico atual para uma cidade especificada.

  Args:
      city (str): O nome da cidade (por exemplo, "Nova York", "Londres", "Tóquio").

  Returns:
      dict: Um dicionário contendo as informações meteorológicas.
            Inclui uma chave 'status' ('success' ou 'error').
            Se 'success', inclui uma chave 'report' com detalhes do tempo.
            Se 'error', inclui uma chave 'error_message'.
  """
  print(f"--- Ferramenta: get_weather chamada para a cidade: {city} ---") # Registra a execução da ferramenta
  city_normalized = city.lower().replace(" ", "") # Normalização básica

  # Dados meteorológicos simulados (mock)
  mock_weather_db = {
      "novayork": {"status": "success", "report": "O tempo em Nova York está ensolarado com uma temperatura de 25°C."},
      "londres": {"status": "success", "report": "Está nublado em Londres com uma temperatura de 15°C."},
      "tokyo": {"status": "success", "report": "Tóquio está com chuva fraca e uma temperatura de 18°C."},
      "goiania": {"status": "success", "report": "Goiânia voltou a chover recentemente e está com uma agradável temperatura de 22°C."},
  }

  if city_normalized in mock_weather_db:
      return mock_weather_db[city_normalized]
  else:
      return {"status": "error", "error_message": f"Desculpe, não tenho informações meteorológicas para '{city}'."}

# Exemplo de uso da ferramenta (teste opcional)
print(get_weather("Nova York"))
print(get_weather("Goiania"))

--- Ferramenta: get_weather chamada para a cidade: Nova York ---
{'status': 'success', 'report': 'O tempo em Nova York está ensolarado com uma temperatura de 25°C.'}
--- Ferramenta: get_weather chamada para a cidade: Goiania ---
{'status': 'success', 'report': 'Goiânia voltou a chover recentemente e está com uma agradável temperatura de 22°C.'}


**2. Defina o Agente (`weather_agent`)**

Agora, vamos criar o **Agente** em si. Um `Agent` no ADK orquestra a interação entre o usuário, a LLM e as ferramentas disponíveis.

Nós o configuramos com vários parâmetros-chave:

*   `name`: Um identificador único para este agente (por exemplo, "weather\_agent\_v1").
*   `model`: Especifica qual LLM usar (por exemplo, `MODEL_GEMINI_2_0_FLASH`). Começaremos com um modelo Gemini específico.
*   `description`: Um resumo conciso do propósito geral do agente. Isso se torna crucial mais tarde, quando outros agentes precisarem decidir se devem delegar tarefas para *este* agente.
*   `instruction`: Orientação detalhada para a LLM sobre como se comportar, sua persona, seus objetivos e, especificamente, *como e quando* utilizar suas `tools` (ferramentas) atribuídas.
*   `tools`: Uma lista contendo as funções Python reais das ferramentas que o agente tem permissão para usar (por exemplo, `[get_weather]`).

**Melhor Prática:** Forneça prompts de `instruction` (instrução) claros e específicos. Quanto mais detalhadas as instruções, melhor a LLM pode entender seu papel e como usar suas ferramentas de forma eficaz. Seja explícito sobre o tratamento de erros, se necessário.

**Melhor Prática:** Escolha valores descritivos para `name` (nome) e `description` (descrição). Eles são usados internamente pelo ADK e são vitais para recursos como a delegação automática (abordada mais tarde).

In [ ]:
# @title Defina o Agente de Clima
# Use uma das constantes de modelo definidas anteriormente
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Começando com Gemini

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Pode ser uma string para o Gemini ou um objeto LiteLlm
    description="Fornece informações meteorológicas para cidades específicas.",
    instruction="Você é um assistente de clima prestativo. "
                "Quando o usuário perguntar sobre o clima em uma cidade específica, "
                "use a ferramenta 'get_weather' para encontrar a informação. "
                "Se a ferramenta retornar um erro, informe o usuário educadamente. "
                "Se a ferramenta for bem-sucedida, apresente o boletim meteorológico de forma clara.",
    tools=[get_weather], # Passe a função diretamente
)

print(f"Agente '{weather_agent.name}' criado usando o modelo '{AGENT_MODEL}'.")

Agente 'weather_agent_v1' criado usando o modelo 'gemini-2.0-flash'.



---
**3. Configurar o Runner e o Session Service**

Para gerenciar as conversas e executar o agente, precisamos de mais dois componentes:

*   `SessionService`: Responsável por gerenciar o histórico de conversas e o estado para diferentes usuários e sessões. O `InMemorySessionService` é uma implementação simples que armazena tudo na memória, adequada para testes e aplicações simples. Ele mantém o registro das mensagens trocadas. Exploraremos a persistência de estado mais a fundo no Passo 4.
*   `Runner`: O motor que orquestra o fluxo de interação. Ele recebe a entrada do usuário, a direciona para o agente apropriado, gerencia as chamadas para a LLM e as ferramentas com base na lógica do agente, lida com as atualizações de sessão através do `SessionService` e gera eventos que representam o progresso da interação.

In [ ]:
# @title Configurar o Session Service e o Runner

# --- Gerenciamento de Sessão ---
# Conceito-Chave: O SessionService armazena o histórico e o estado da conversa.
# O InMemorySessionService é um armazenamento simples e não persistente para este tutorial.
session_service = InMemorySessionService()

# Defina constantes para identificar o contexto da interação
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Usando um ID fixo para simplificar

# Crie a sessão específica onde a conversa acontecerá
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Sessão criada: App='{APP_NAME}', Usuário='{USER_ID}', Sessão='{SESSION_ID}'")

# --- Runner ---
# Conceito-Chave: O Runner orquestra o ciclo de execução do agente.
runner = Runner(
    agent=weather_agent, # O agente que queremos executar
    app_name=APP_NAME,   # Associa as execuções ao nosso aplicativo
    session_service=session_service # Usa nosso gerenciador de sessão
)
print(f"Runner criado para o agente '{runner.agent.name}'.")

Sessão criada: App='weather_tutorial_app', Usuário='user_1', Sessão='session_001'
Runner criado para o agente 'weather_agent_v1'.


---
**4. Interaja com o Agente**

Precisamos de uma maneira de enviar mensagens ao nosso agente e receber suas respostas. Como as chamadas à LLM e a execução de ferramentas podem levar tempo, o `Runner` do ADK opera de forma assíncrona.

Vamos definir uma função auxiliar `async` (`call_agent_async`) que:

1.  Recebe uma string com a consulta do usuário.
2.  A empacota no formato `Content` do ADK.
3.  Chama `runner.run_async`, fornecendo o contexto de usuário/sessão e a nova mensagem.
4.  Itera através dos **Eventos** (Events) gerados pelo runner. Os eventos representam os passos na execução do agente (por exemplo, chamada de ferramenta solicitada, resultado da ferramenta recebido, pensamento intermediário da LLM, resposta final).
5.  Identifica e imprime o evento de **resposta final** usando `event.is_final_response()`.

**Por que `async`?** As interações com LLMs e, potencialmente, com ferramentas (como APIs externas) são operações limitadas por I/O (entrada/saída). O uso de `asyncio` permite que o programa lide com essas operações de forma eficiente, sem bloquear a execução.

In [ ]:
# @title Defina a Função de Interação com o Agente

from google.genai import types # Para criar Conteúdo/Partes de mensagem

async def call_agent_async(query: str, runner, user_id, session_id):
  """Envia uma consulta ao agente e imprime a resposta final."""
  print(f"\n>>> Consulta do Usuário: {query}")

  # Prepare a mensagem do usuário no formato ADK
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "O agente não produziu uma resposta final." # Padrão

  # Conceito-Chave: run_async executa a lógica do agente e gera Eventos (Events).
  # Iteramos através dos eventos para encontrar a resposta final.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # Você pode descomentar a linha abaixo para ver *todos* os eventos durante a execução
      # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Conteúdo: {event.content}")

      # Conceito-Chave: is_final_response() marca a mensagem de conclusão do turno.
      if event.is_final_response():
          if event.content and event.content.parts:
              # Assumindo uma resposta em texto na primeira parte
              final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Lide com erros/escalonamentos potenciais
              final_response_text = f"Agente escalonou: {event.error_message or 'Nenhuma mensagem específica.'}"
          # Adicione mais verificações aqui se necessário (por exemplo, códigos de erro específicos)
          break # Pare de processar eventos assim que a resposta final for encontrada

  print(f"<<< Resposta do Agente: {final_response_text}")

---
**5. Execute a Conversa**

Finalmente, vamos testar nossa configuração enviando algumas consultas ao agente. Nós encapsulamos nossas chamadas `async` em uma função `async` principal e a executamos usando `await`.

Observe a saída:

*   Veja as consultas do usuário.
*   Note os logs `--- Ferramenta: get_weather chamada... ---` quando o agente usa a ferramenta.
*   Observe as respostas finais do agente, incluindo como ele lida com o caso em que os dados meteorológicos não estão disponíveis (para Paris).

In [ ]:
# @title Execute a Conversa Inicial

# Precisamos de uma função async para aguardar nosso auxiliar de interação
async def run_conversation():
    await call_agent_async("Como está o tempo em Londres?",
                                 runner=runner,
                                 user_id=USER_ID,
                                 session_id=SESSION_ID)

    await call_agent_async("E em Paris?",
                                 runner=runner,
                                 user_id=USER_ID,
                                 session_id=SESSION_ID) # Esperando a mensagem de erro da ferramenta

    await call_agent_async("Me diga o tempo em Nova York",
                                 runner=runner,
                                 user_id=USER_ID,
                                 session_id=SESSION_ID)

# Execute a conversa usando await em um contexto assíncrono (como Colab/Jupyter)
await run_conversation()

# --- OU ---

# Descomente as linhas a seguir se estiver executando como um script Python padrão (arquivo .py):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"Ocorreu um erro: {e}")


>>> Consulta do Usuário: Como está o tempo em Londres?


--- Ferramenta: get_weather chamada para a cidade: Londres ---
<<< Resposta do Agente: Está nublado em Londres com uma temperatura de 15°C.


>>> Consulta do Usuário: E em Paris?


--- Ferramenta: get_weather chamada para a cidade: Paris ---
<<< Resposta do Agente: Desculpe, não tenho informações meteorológicas para Paris.

>>> Consulta do Usuário: Me diga o tempo em Nova York


--- Ferramenta: get_weather chamada para a cidade: Nova York ---
<<< Resposta do Agente: O tempo em Nova York está ensolarado com uma temperatura de 25°C.




---

Parabéns! Você construiu e interagiu com sucesso com seu primeiro agente ADK. Ele entende a solicitação do usuário, usa uma ferramenta para encontrar informações e responde adequadamente com base no resultado da ferramenta.

No próximo passo, exploraremos como trocar facilmente o Modelo de Linguagem (Language Model) que alimenta este agente.